In [1]:
import pandas as pd
from tqdm import tqdm
import csv

In [2]:
# Сгенерируем колонки
def generate_columns(n, name):
    columns = [name+str(i) for i in range(1, n+1)]
    return columns

In [3]:
# Из общего файла n-грамм выкидываем инфу про хостов
# Остаются только сами последовательности слов*
# *не самих лексем, а их частей речи
def without_host(n, my_columns, file_dir):
    df_all = pd.read_csv(file_dir + f"/all_count_{n}_grams.csv", sep=",")
    df_wth_host = df_all[my_columns + ["total_entries"]]
    return df_wth_host

In [4]:
# Группируем последовательности ЧР 
# Добавляем информацию про число вхождений каждого типа последовательности
# и число различных видов подчинения внутри каждого типа
# записываем в файл
def order_entries_and_reltypes(n, my_columns, file_dir):
    df_wth_host = without_host(n, my_columns, file_dir=file_dir)
    df_grouped = df_wth_host.groupby(my_columns, as_index=False).agg({"total_entries": sum})
    df_grouped["number_of_rel_types"] = df_wth_host.groupby(my_columns, as_index=False)\
                                        .agg({"total_entries": "count"})["total_entries"]
    df_grouped = df_grouped.sort_values(by=["total_entries", "number_of_rel_types"], ascending=[False, False])
    df_grouped = df_grouped.rename(columns={'total_entries': 'total_entries_of_construction'})
    df_grouped.to_csv(file_dir + f"/all_order_entries_and_reltypes_{n}.csv", index=None)

In [5]:
for i in range(3,7):
    order_entries_and_reltypes(i, generate_columns(i, "POS"), "./outcome files/dev_train_test")

In [6]:
for i in range(3,7):
    order_entries_and_reltypes(i, generate_columns(i, "POS"), "./outcome files/dev_train")

## Доли типов подчинения внутри типов n-грамм (по одной связи)

In [36]:
# Добавляем колонку с номерами конструкций
# Соединяем два df по ключам-частям речи
# Дробаем лишнее и переставляем покрасивее
def df_all_for_ratio(n, file_dir):
    df_to_stay = pd.read_csv(file_dir + f"/all_order_entries_and_reltypes_{n}.csv", sep=",")
    df_to_stay["#construction"] = df_to_stay.index
    
    df_all = pd.read_csv(file_dir + f"/all_count_{n}_grams.csv", sep=",")
    df_all = pd.merge(df_all, df_to_stay, on=generate_columns(n, "POS"))

    df_all = df_all.drop(["number_of_rel_types"], axis=1)
    df_all = df_all[["#construction"] + list(df_all.columns.values)[:-1]]
    df_all = df_all.sort_values(generate_columns(n, "POS")).reset_index(drop=True)
    
    return df_all

In [37]:
# Словарик для группировки по конструкциям
def create_agg_dict(n):
    agg_dict = {"total_entries": 'sum', "total_entries_of_construction": 'first', 
                "#construction": 'first'}
    for col in generate_columns(n, "POS"):
        agg_dict[col] = 'first' 
    return agg_dict

In [38]:
# Заполнитель нулями (0 = на это не смотрим)
def place_zero(i, n, constr_ratio):
    for i_n in  range (1, n + 1):
        if i_n != i:
            constr_ratio[f"#host{i_n}"] = 0
            constr_ratio[f"POShost{i_n}"] = 0

In [39]:
# Вытаксиваем отношения для i-го слова в конструкции
def ratio_from_construction(i, n, constr, agg_dict): 
    constr_ratio = constr.groupby([f"#host{i}", f"POShost{i}"], as_index=False).agg(agg_dict)

    constr_ratio["ratio"] = round(constr_ratio["total_entries"] / 
                                  constr_ratio["total_entries_of_construction"], 3)
    constr_ratio = constr_ratio.sort_values(by="ratio", ascending=False).reset_index(drop=True)
    place_zero(i, n, constr_ratio)
    constr_ratio = constr_ratio.drop(["total_entries_of_construction"], axis=1)
    return constr_ratio

In [40]:
# Создаем csv файл для дальнейшей записи по кусочкам
def init_csv(n, file_dir):
    with open(file_dir + f"/all_with_ratios_{n}.csv", 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',')
        csvwriter.writerow(right_col_order(n))

In [41]:
# Правильный порядок для записи
def right_col_order(n):
    right_order = ["#construction"]
    for i in range(n):
        right_order.extend([generate_columns(n, "POS")[i], 
                            generate_columns(n, "#host")[i], 
                            generate_columns(n, "POShost")[i]])
    right_order.extend(["ratio", "total_entries"])
    return right_order

In [42]:
# Записываем в csv кусочек конструкций
def get_result(n, ratio_from_construction, file_dir):
    result = ratio_from_construction[right_col_order(n)].reset_index(drop=True)
    with open(file_dir + f"/all_with_ratios_{n}.csv", 'a') as csvadd:
        result.to_csv(csvadd, header=False, index=None, line_terminator='\n')

In [43]:
# constrs_num -- список номеров конструкций
# Общая функция (! очень долго выполняется, и чем больше n, тем дольше)
def ratios_of_reltypes(n, file_dir):
    init_csv(n, file_dir=file_dir)

    df_all = df_all_for_ratio(n, file_dir=file_dir)
    constrs_num = pd.unique(df_all["#construction"]).tolist()
    agg_dict = create_agg_dict(n)

    for num in tqdm(constrs_num):
        constr = df_all[df_all["#construction"] == num].reset_index()
        for i in range(1, n + 1):
            get_result(n, ratio_from_construction(i, n, constr, agg_dict), file_dir=file_dir)

In [45]:
ratios_of_reltypes(3, "./outcome files/dev_train_test")
#ratios_of_reltypes(3, "./outcome files/dev_train")

100%|██████████████████████████████████████████████████████████████████████████████| 2874/2874 [04:13<00:00, 11.34it/s]


In [16]:
# ratios_of_reltypes(4, "./outcome files/dev_train_test")
# ratios_of_reltypes(4, "./outcome files/dev_train")

In [17]:
# ratios_of_reltypes(5, "./outcome files/dev_train_test")
# ratios_of_reltypes(5, "./outcome files/dev_train")

In [18]:
# ratios_of_reltypes(6, "./outcome files/dev_train_test")
# ratios_of_reltypes(6, "./outcome files/dev_train")

## Доли типов подчинения внутри типов n-грамм (все связи)

In [13]:
def ratios_of_reltypes_all(n, file_dir):
    df_all = df_all_for_ratio(n, file_dir=file_dir)
    df_all["ratio"] = round(df_all["total_entries"] / df_all["total_entries_of_construction"], 3)
    df_all = df_all.sort_values(generate_columns(n, "POS")).reset_index(drop=True)
    df_all.to_csv(file_dir + f"/all_full_with_ratios_{n}.csv", index=None)

In [14]:
for i in range(3, 7):
    ratios_of_reltypes_all(i, "./outcome files/dev_train_test")

In [15]:
for i in range(3, 7):
    ratios_of_reltypes_all(i, "./outcome files/dev_train")